In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
#pd.set_option('display.max_columns', None)

In [3]:
usco2000 = pd.read_csv('usco2000.csv', dtype = str)
usco2005 = pd.read_csv('usco2005.csv', dtype = str)
usco2010 = pd.read_csv('usco2010.csv', dtype = str)
usco2015 = pd.read_csv('usco2015v2.0.csv', dtype = str)
usco2015.columns = usco2015.loc[0,:]
usco2015 = usco2015.iloc[1:]

Drop fully null columns, drop Puerto Rico and Virgin Islands

In [4]:
usco2000 = usco2000.dropna(subset = ['STATEFIPS'])
usco2005 = usco2005.dropna(subset = ['STATEFIPS'])
usco2010 = usco2010.dropna(subset = ['STATEFIPS'])
usco2015 = usco2015.dropna(subset = ['STATEFIPS'])

In [5]:
usco2000 = usco2000[usco2000['STATEFIPS'].astype(int) <= 56]
usco2005 = usco2005[usco2005['STATEFIPS'].astype(int) <= 56]
usco2010 = usco2010[usco2010['STATEFIPS'].astype(int) <= 56]
usco2015 = usco2015[usco2015['STATEFIPS'].astype(int) <= 56]

Manually listed columns that appear differently in the four datasets

In [6]:
#columns that appear in all datasets, written the same way each time
in_all = ['STATE', 'STATEFIPS', 'COUNTYFIPS', 'FIPS', 'TP-TotPop', 'PS-TOPop', 'PS-WGWFr', 'PS-WSWFr', 'PS-WFrTo', 
          'DO-SSPop','DO-WGWFr','DO-WSWFr','DO-WFrTo', 'DO-PSDel', 'IN-WGWFr','IN-WGWSa','IN-WGWTo','IN-WSWFr','IN-WSWSa',
          'IN-WSWTo','IN-WFrTo','IN-WSaTo','IN-Wtotl', 'MI-WGWFr', 'MI-WGWSa', 'MI-WGWTo', 'MI-WSWFr', 'MI-WSWSa', 
          'MI-WSWTo', 'MI-WFrTo', 'MI-WSaTo', 'MI-Wtotl', 'PO-WSWFr', 'PO-WSWSa', 'PO-WSWTo', 'TO-WGWFr', 'TO-WGWSa',
          'TO-WGWTo', 'TO-WSWFr', 'TO-WSWSa', 'TO-WSWTo', 'TO-WFrTo', 'TO-WSaTo', 'TO-WTotl'
         ]

#only usco2000 does not have these columns
missing_2000 = ['PS-GWPop', 'PS-SWPop', 'PS-WGWSa', 'PS-WGWTo', 'PS-WSWSa', 'PS-WSWTo', 'PS-WSaTo','PS-Wtotl',
                'IC-WGWFr', 'IC-WSWFr', 'IC-WFrTo', 'IC-IrSpr', 'IC-IrMic', 'IC-IrSur', 'IC-IrTot', 'IG-WGWFr', 
                'IG-WSWFr', 'IG-WFrTo', 'IG-IrSpr', 'IG-IrMic', 'IG-IrSur', 'IG-IrTot', 'PT-WGWSa', 'PT-WGWTo', 
                'PT-Power', 'PO-WGWFr', 'PO-WGWSa', 'PO-WGWTo', 'PO-WFrTo', 'PO-WSaTo', 'PO-Wtotl', 'PO-Power',
                'PC-WGWSa', 'PC-WGWTo', 'PC-Power', 'DO-WDelv', 'DO-PSDel', 'IR-WSWFr'
               ]

#both usco2000 and usco2005 do not have these columns.
missing_2000_and_2005 = ['DO-SSPCp', 'DO-PSPCp', 'AQ-WGWSa', 'AQ-WGWTo', 'AQ-WSWSa', 'AQ-WSWTo', 'AQ-WSaTo', 'AQ-Wtotl']

#columns that only appear in usco2015
only_in_2015 = ['IR-RecWW', 'IR-CUsFr', 'IC-RecWW', 'IC-CUsFr', 'IG-RecWW', 'IG-CUsFr', 'PT-RecWW', 'PT-PSDel', 
                'PT-CUsFr','PT-CUsSa','PT-CUTot', 'PO-RecWW', 'PO-PSDel', 'PO-CUsFr', 'PO-CUsSa', 'PO-CUTot', 'PC-RecWW', 
                'PC-PSDel','PC-CUsFr', 'PC-CUsSa', 'PC-CUTot', 'TO-CUsFrPartial', 'TO-CUsSaPartial', 'TO-CUTotPartial',
               ]

#change these column names in the usco2000 dataset to match with other dataets
dict_2000_change = {'IT-IrSpr':'IR-IrSpr', 'IT-IrMic': 'IR-IrMic', 'IT-IrSur': 'IR-IrSur', 'IT-IrTot': 'IR-IrTot',
                    'IT-WGWFr': 'IR-WGWFr', 'IT-WSWFr':'IR-WSWFr', 'IT-WFrTo': 'IR-WFrTo', 'LS-WGWFr':'LI-WGWFr',
                    'LS-WSWFr': 'LI-WSWFr', 'LS-WFrTo': 'LI-WFrTo', 'LA-WGWFr': 'AQ-WGWFr', 'LA-WSWFr':'AQ-WSWFr',
                    'LA-WFrTo':'AQ-WFrTo', 'PE-WGWFr':'PC-WGWFr', 'PE-WSWFr':'PC-WSWFr', 'PE-WSWSa':'PC-WSWSa', 
                    'PE-WSWTo':'PC-WSWTo', 'PE-WFrTo':'PC-WFrTo', 'PE-WSaTo':'PC-WSaTo', 'PE-Wtotl':'PC-Wtotl',
                    'TO-WTotl': 'TO-Wtotl'
                   }

#change these column names in the usco2005 dataset to match with other dataets
dict_2005_change = {'LS-WGWFr':'LI-WGWFr', 'LS-WSWFr': 'LI-WSWFr', 'LS-WFrTo': 'LI-WFrTo', 'LA-WGWFr': 'AQ-WGWFr', 
                    'LA-WSWFr':'AQ-WSWFr', 'LA-WFrTo':'AQ-WFrTo', 'DO-TOTAL ': 'DO-WDelv', 'PO-WTotl': 'PO-Wtotl',
                    'PC-WTotl':'PC-Wtotl', 'TO-WTotl': 'TO-Wtotl'
                   }

#change this column name in the usco2010 dataset to match with other dataets
dict_2010_change = {'DO-TOTAL ':'DO-WDelv', 'AQ-WTotl': 'AQ-Wtotl', 'PO-WTotl': 'PO-Wtotl', 'PC-WTotl':'PC-Wtotl',
                    'TO-WTotl': 'TO-Wtotl'}

In [7]:
usco2000 = usco2000.rename(columns=dict_2000_change)
usco2005 = usco2005.rename(columns=dict_2005_change)
usco2010 = usco2010.rename(columns=dict_2010_change)
usco2015 = usco2015.rename(columns = {'DO-WDelv ': 'DO-WDelv'})

## Replace missing values to np.nan

In [8]:
usco2015.replace('--', value = np.NaN, inplace = True)

Filter out columns we do not want

In [9]:
power_out = ['PT-Power', 'PO-Power', 'PC-Power']
pop_out = ['PS-GWPop', 'PS-SWPop',]
irrigation = ['IC-WGWFr', 'IC-WSWFr', 'IC-WFrTo', 'IC-RecWW', 'IC-CUsFr', 'IC-IrSpr', 
              'IC-IrMic', 'IC-IrSur', 'IC-IrTot', 'IG-WGWFr', 'IG-WSWFr', 'IG-WFrTo', 
              'IG-RecWW', 'IG-CUsFr', 'IG-IrSpr', 'IG-IrMic', 'IG-IrSur', 'IG-IrTot']
totals = []

Run below block to remove possibly redundant "totals" colums. We removed totals for the model in the report.

In [10]:
totals = ['PS-TOPop', 'PS-WGWTo', 'PS-Wtotl', 'PS-WGWTo', 'PS-WSWTo', 'PS-Wtotl', 'IN-WGWTo', 'IN-WSWTo', 
          'IN-Wtotl', 'IC-IrTot', 'IG-IrTot', 'AQ-WGWTo', 'AQ-WSWTo', 'AQ-Wtotl', 'MI-WGWTo', 'MI-WSWTo', 
          'MI-Wtotl', 'PT-WGWTo', 'PT-WSWTo', 'PT-Wtotl', 'PO-WGWTo', 'PO-WSWTo', 'PO-Wtotl', 'PC-WGWTo', 
          'PC-WSWTo', 'PC-Wtotl', 'TO-WGWTo', 'TO-WSWTo', 'TO-Wtotl'
         ]

Remove columns here

In [11]:
remove_columns = missing_2000_and_2005 + only_in_2015 + pop_out + irrigation + power_out + ['YEAR'] + totals 
cols_2015 = list(usco2015.columns)
columns_interest = [col for col in cols_2015 if col not in remove_columns]

See which counties are not represented in all four datasets. These include newly designated or former counties. We will drop them.

In [12]:
fips_count_dict = {}
for df in [(usco2000, 2000), (usco2005, 2005), (usco2010,2010), (usco2015,2015)]:
    for i, row in df[0].iterrows():
        fips = row['FIPS']
        if fips in fips_count_dict:
            fips_count_dict[fips].append(df[1])
        else:
            fips_count_dict[fips] = [df[1]]
            
partial_counties = []
for key, val in fips_count_dict.items():
    if len(val)<4:
        partial_counties.append(key)

#### Calculate slopes and means for each data column. Reformat into single dataframe

In [ ]:
fips_list = list(fips_count_dict.keys())
df_list = [usco2000, usco2005, usco2010, usco2015]
new_dict = {}
c = 0
for i, row in usco2015.iterrows():
    c +=1
    if c % 100 == 0:
        print(c)
        
    fips = row['FIPS']
    if fips in partial_counties:
        continue
        
    county_dict = {}
    for col in columns_interest:
        if col in ['STATE','STATEFIPS','COUNTY','COUNTYFIPS','FIPS']:
            county_dict[col] = row[col]
            continue
        
        #calculate the slope of the data column across the four datasets, and find the mean value.
        data = []
        temp_count = 0
        for df in df_list:
            
            if col not in list(df.columns):
                continue
            
            fips_data = df[df['FIPS'] == fips]
            fips_data = fips_data.reset_index()
            col_val = fips_data.loc[0, col]
            if pd.isna(col_val) == True:
                continue
            
            data.append(float(col_val))
            temp_count +=1
        
        if temp_count == 1:
            print(fips, col)
            
        #run sklearn linear regression
        x = np.array(list(range(temp_count))).reshape(-1,1)
        y = np.array(data).reshape(-1,1)
        linreg = LinearRegression().fit(x, data)            
        slope = linreg.coef_[0]
        
        county_dict[col + '_mean'] = np.mean(data)
        county_dict[col + '_slope'] = slope
    
    new_dict[fips] = county_dict

In [15]:
export_df = pd.DataFrame(new_dict).T

export the dataframe

In [63]:
#export_df.to_csv('water-use_cleaned.csv', index = False)
export_df.to_csv('water-use_cleaned_totals-removed.csv', index = False)